In [1]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

In [2]:
import sqlContext.implicits._

In [3]:
import org.apache.spark.sql.types._

In [4]:
import org.apache.spark.sql.functions._

In [5]:
val cases_data = spark.read.format("csv").option("header", "true").option("quote", "\"").option("escape", "\"").option("mode", "DROPMALFORMED").option("inferSchema", "true").load("hdfs://sandbox.hortonworks.com:8020/tmp/cases.csv")

In [6]:
val bc_cases_data = sc.broadcast(cases_data)

In [9]:
cases_data.dtypes

Array((case_row_id,IntegerType), (case_number,StringType), (pacer_id,IntegerType), (case_name,StringType), (court_name,StringType), (assigned_to,StringType), (referred_to,StringType), (case_cause,StringType), (jurisdictional_basis,StringType), (demand,StringType), (jury_demand,StringType), (lead_case,StringType), (related_case,StringType), (settlement,StringType), (date_filed,TimestampType), (date_closed,TimestampType), (date_last_filed,TimestampType))

In [10]:
val docs_data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("quote", "\"").option("escape", "\"").load("hdfs://sandbox.hortonworks.com:8020/tmp/documents.csv")

In [21]:
cases_data.groupBy("case_name").count().show()

+--------------------+-----+
|           case_name|count|
+--------------------+-----+
|Turk v. WRR North...|    1|
|Raymedica, Inc. v...|    1|
|Wilson Tool Intl ...|    1|
|CMB Enterprises, ...|    1|
|USL Products Inc ...|    1|
|Globefill Incorpo...|    1|
|AAB Building Syst...|    1|
|Aspex Eyewear, In...|    1|
|A & A Global v. L...|    1|
|RTC Ind Inc v. HM...|    1|
|Intellectual Prop...|    1|
|Randolph-Rand Cor...|    1|
|Gardner Pie Compa...|    1|
|In Re: Singer Com...|    1|
|VLT Corporation, ...|    1|
|Divine Tech Ventu...|    1|
|Gillette Company,...|    1|
|Eisai Co., Ltd., ...|    1|
|Hoefer v. Sony Co...|    1|
|Evenflo Company, ...|    1|
+--------------------+-----+
only showing top 20 rows



In [11]:
docs_data.dtypes

Array((case_row_id,IntegerType), (case_number,StringType), (doc_count,IntegerType), (attachment,IntegerType), (date_filed,TimestampType), (long_description,StringType), (doc_number,StringType), (short_description,StringType), (upload_date,TimestampType))

In [12]:
docs_data.count()

5186344

In [13]:
val names_data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("quote", "\"").option("escape", "\"").load("hdfs://sandbox.hortonworks.com:8020/tmp/names.csv")

In [14]:
names_data.dtypes

Array((case_row_id,IntegerType), (case_number,StringType), (party_row_count,IntegerType), (party_type,StringType), (name_row_count,IntegerType), (name,StringType))

In [22]:
names_data.take(5)

Array([1,0:79-cv-06704-JCP,1,Plaintiff,1,Burroghs Wellcome Co.], [1,0:79-cv-06704-JCP,2,Defendant,2,Generix Drug Corp.], [3,0:83-cv-06860-JAG,3,Plaintiff,3,Kenneth R. Cornwall], [3,0:83-cv-06860-JAG,4,Defendant,4,U. S. COnstruction Manufacturing, Inc.], [4,0:84-cv-06456-KLR,5,Plaintiff,5,Monte Carlo Hairpieces, Inc.])

In [29]:
val defendants = names_data.filter("party_type=='Defendant'").show()

+-----------+-----------------+---------------+----------+--------------+--------------------+
|case_row_id|      case_number|party_row_count|party_type|name_row_count|                name|
+-----------+-----------------+---------------+----------+--------------+--------------------+
|          1|0:79-cv-06704-JCP|              2| Defendant|             2|  Generix Drug Corp.|
|          3|0:83-cv-06860-JAG|              4| Defendant|             4|U. S. COnstructio...|
|          4|0:84-cv-06456-KLR|              7| Defendant|             7|On-Rite Hairpiece...|
|          4|0:84-cv-06456-KLR|              8| Defendant|             8|    Andrew O. Wright|
|          5|0:84-cv-06726-WMH|             11| Defendant|            11|Atlantic Federal ...|
|          5|0:84-cv-06726-WMH|             12| Defendant|            12|Ozark Financial, ...|
|          5|0:84-cv-06726-WMH|             13| Defendant|            13|Government Employ...|
|          5|0:84-cv-06726-WMH|             14| De

In [28]:
val plaintiffs = names_data.filter("party_type=='Plaintiff'").show()

+-----------+-----------------+---------------+----------+--------------+--------------------+
|case_row_id|      case_number|party_row_count|party_type|name_row_count|                name|
+-----------+-----------------+---------------+----------+--------------+--------------------+
|          1|0:79-cv-06704-JCP|              1| Plaintiff|             1|Burroghs Wellcome...|
|          3|0:83-cv-06860-JAG|              3| Plaintiff|             3| Kenneth R. Cornwall|
|          4|0:84-cv-06456-KLR|              5| Plaintiff|             5|Monte Carlo Hairp...|
|          4|0:84-cv-06456-KLR|              6| Plaintiff|             6|     James L. Waters|
|          5|0:84-cv-06726-WMH|              9| Plaintiff|             9|Monaco Del, Rocco...|
|          5|0:84-cv-06726-WMH|             10| Plaintiff|            10|  Frances Monaco Del|
|          6|0:85-cv-06225-TES|             18| Plaintiff|            18|Contour Chair Lou...|
|          6|0:85-cv-06225-TES|             19| Pl

In [10]:
//val doc_dates = docs_data.groupBy("date_filed").agg(count("*") as "date_counts").orderBy($"date_counts" desc)

In [11]:
//doc_dates.show()

+--------------------+-----------+
|          date_filed|date_counts|
+--------------------+-----------+
|                null|      47319|
|2015-11-30 00:00:...|       2568|
|2012-12-21 00:00:...|       2402|
|2013-12-20 00:00:...|       2129|
|2014-11-24 00:00:...|       2011|
|2013-01-15 00:00:...|       2001|
|2014-03-31 00:00:...|       1998|
|2013-11-22 00:00:...|       1997|
|2012-09-28 00:00:...|       1990|
|2013-02-19 00:00:...|       1975|
|2014-01-06 00:00:...|       1961|
|2014-01-17 00:00:...|       1947|
|2013-02-15 00:00:...|       1943|
|2011-10-11 00:00:...|       1920|
|2011-11-14 00:00:...|       1916|
|2012-09-10 00:00:...|       1915|
|2013-02-01 00:00:...|       1906|
|2014-01-27 00:00:...|       1906|
|2013-12-09 00:00:...|       1906|
|2011-10-21 00:00:...|       1905|
+--------------------+-----------+
only showing top 20 rows



In [12]:
val sh_desc = docs_data.groupBy("short_description").agg(count("*") as "short_count").orderBy($"short_count" desc)

In [13]:
sh_desc.show()

+--------------------+-----------+
|   short_description|short_count|
+--------------------+-----------+
|                null|    4834609|
|               Order|      18478|
|      Notice (Other)|      12050|
|              Letter|      11710|
|Certificate of Se...|       9619|
|Notice to Take De...|       5589|
|         Stipulation|       5346|
|Notice of Attorne...|       5248|
|          Transcript|       5232|
|   Notice of Service|       5167|
|         Declaration|       4939|
|Motion for Leave ...|       4123|
|Notice of Appearance|       3820|
| Answer to Complaint|       3517|
|Declaration in Su...|       3381|
|Application to Ap...|       3234|
|Reply to Response...|       3222|
|Motion for Miscel...|       3199|
|Summons Returned ...|       3196|
|Answer to Counter...|       3167|
+--------------------+-----------+
only showing top 20 rows



In [14]:
docs_data.filter($"long_description".isNotNull).select("long_description").show()

+--------------------+
|    long_description|
+--------------------+
|COPY OF PAPER DOC...|
|CASE CLOSED. Case...|
|COPY OF PAPER DOC...|
|Case closed (Form...|
|Case reopened (Fo...|
|Case closed (Form...|
|FINAL JUDGMENT fo...|
|COPY OF PAPER DOC...|
|COPY OF PAPER DOC...|
|D.E. #1 should be...|
|Case closed (cp, ...|
|ORDER DISMISSING ...|
|COPY OF PAPER DOC...|
|COPY OF PAPER DOC...|
|COPY OF PAPER DOC...|
|COPY OF PAPER DOC...|
|COPY OF PAPER DOC...|
|Case closed (Form...|
|COPY OF PAPER DOC...|
|COPY OF PAPER DOC...|
+--------------------+
only showing top 20 rows



In [15]:
//val long_counts = docs_data.filter($"long_description".isNotNull).groupBy("long_description").agg(count("*") as "long_count").orderBy($"long_count" desc)

In [16]:
//long_counts.show()

+--------------------+----------+
|    long_description|long_count|
+--------------------+----------+
|Case assigned to ...|       110|
|In accordance wit...|       103|
|Order reassigning...|       103|
|ORDER REGARDING T...|       101|
|ORDER; Status Con...|        97|
|Report to Commiss...|        91|
|NOTICE by TLI Com...|        90|
|NOTICE of Attorne...|        87|
|NOTICE of Attorne...|        86|
|Notice of Filing ...|        86|
|MEMORANDUM OPINIO...|        86|
|CORPORATE DISCLOS...|        86|
|Case reassigned t...|        84|
|ORDER OF CONSOLID...|        81|
|Order reassigning...|        77|
|In accordance wit...|        76|
|In accordance wit...|        75|
|Case assigned to ...|        74|
|ORDER REASSIGNING...|        74|
|Sealed Document. ...|        74|
+--------------------+----------+
only showing top 20 rows



In [19]:
//val upload_dates = docs_data.groupBy("upload_date").count()

In [20]:
//upload_dates.show()

+--------------------+-----+
|         upload_date|count|
+--------------------+-----+
|2010-11-02 00:00:...|    9|
|2012-07-11 00:00:...|   24|
|2010-04-19 00:00:...|   28|
|2011-05-25 00:00:...|   13|
|2015-07-14 00:00:...|    8|
|2013-10-31 00:00:...|   23|
|2010-10-22 00:00:...|    5|
|2010-03-25 00:00:...|    9|
|2011-10-10 00:00:...|    7|
|2012-10-20 00:00:...|    8|
|2014-08-04 00:00:...|   19|
|2015-04-26 00:00:...|   17|
|2007-10-01 00:00:...|   12|
|2009-06-10 00:00:...|    4|
|2007-07-24 00:00:...|    1|
|2010-05-03 00:00:...|   13|
|2009-10-15 00:00:...|   14|
|2012-01-22 00:00:...|    4|
|2010-06-26 00:00:...|    1|
|2012-12-17 00:00:...|    7|
+--------------------+-----+
only showing top 20 rows



In [ ]:
//val long_desc = docs_data.filter($"long_description".isNotNull).select("long_description").rdd.map(x=>x(0)).collect()